In [1]:
input_owl_string = """<?xml version="1.0"?>
<rdf:RDF xmlns="http://edamontology.org/"
     xml:base="http://edamontology.org/"
     xmlns:dc="http://purl.org/dc/elements/1.1/"
     xmlns:dcterms="http://purl.org/dc/terms/"
     xmlns:owl="http://www.w3.org/2002/07/owl#"
     xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
     xmlns:skos="http://www.w3.org/2004/02/skos/core#"
     xmlns:xml="http://www.w3.org/XML/1998/namespace"
     xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
     xmlns:doap="http://usefulinc.com/ns/doap#"
     xmlns:foaf="http://xmlns.com/foaf/0.1/"
     xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
     xmlns:oboInOwl="http://www.geneontology.org/formats/oboInOwl#"
     xmlns:oboLegacy="http://purl.obolibrary.org/obo/">
    <owl:Ontology rdf:about="http://edamontology.org">
        <next_id>4040</next_id>
    </owl:Ontology>
    


    <!-- http://edamontology.org/data_0007 -->

    <owl:Class rdf:about="http://edamontology.org/data_0007">
        <rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#DeprecatedClass"/>
        <created_in>beta12orEarlier</created_in>
        <obsolete_since>beta12orEarlier</obsolete_since>
        <oldParent rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
        <oboInOwl:hasDefinition>A bioinformatics package or tool, e.g. a standalone application or web service.</oboInOwl:hasDefinition>
        <oboInOwl:inSubset rdf:resource="http://edamontology.org/obsolete"/>
        <oboInOwl:replacedBy rdf:resource="http://edamontology.org/data_0958"/>
        <rdfs:label>Tool</rdfs:label>
        <owl:deprecated rdf:datatype="http://www.w3.org/2001/XMLSchema#boolean">true</owl:deprecated>
    </owl:Class>
    


    <!-- http://edamontology.org/data_0581 -->

    <owl:Class rdf:about="http://edamontology.org/data_0581">
        <rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#DeprecatedClass"/>
        <created_in>beta12orEarlier</created_in>
        <obsolete_since>beta12orEarlier</obsolete_since>
        <oldParent rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
        <oboInOwl:hasDefinition>A digital data archive typically based around a relational model but sometimes using an object-oriented, tree or graph-based model.</oboInOwl:hasDefinition>
        <oboInOwl:inSubset rdf:resource="http://edamontology.org/obsolete"/>
        <oboInOwl:replacedBy rdf:resource="http://edamontology.org/data_0957"/>
        <rdfs:label>Database</rdfs:label>
        <owl:deprecated rdf:datatype="http://www.w3.org/2001/XMLSchema#boolean">true</owl:deprecated>
    </owl:Class>

    <!-- http://www.w3.org/2002/07/owl#DeprecatedClass -->

    <owl:Class rdf:about="http://www.w3.org/2002/07/owl#DeprecatedClass"/>
</rdf:RDF>
"""

In [2]:
from rdflib import Graph, URIRef, RDF, OWL, Namespace

from rdflib.plugins.serializers.rdfxml import XMLSerializer

from io import BytesIO

import xml.dom.minidom

def create_owl_elements(input_owl_string):
    try:
         # Define custom namespace prefixes
        edam_namespace = Namespace("http://edamontology.org/")
        dc_namespace = Namespace("http://purl.org/dc/elements/1.1/")
        dcterms_namespace = Namespace("http://purl.org/dc/terms/")
        owl_namespace = Namespace("http://www.w3.org/2002/07/owl#")
        rdf_namespace = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
        skos_namespace = Namespace("http://www.w3.org/2004/02/skos/core#")
        xml_namespace = Namespace("http://www.w3.org/XML/1998/namespace")
        xsd_namespace = Namespace("http://www.w3.org/2001/XMLSchema#")
        doap_namespace = Namespace("http://usefulinc.com/ns/doap#")
        foaf_namespace = Namespace("http://xmlns.com/foaf/0.1/")
        rdfs_namespace = Namespace("http://www.w3.org/2000/01/rdf-schema#")
        oboInOwl_namespace = Namespace("http://www.geneontology.org/formats/oboInOwl#")
        oboLegacy_namespace = Namespace("http://purl.obolibrary.org/obo/")

        # Load the RDF graph from the input string with explicit namespaces
        namespaces = {
            "": edam_namespace,
            "dc": dc_namespace,
            "dcterms": dcterms_namespace,
            "owl": owl_namespace,
            "rdf": rdf_namespace,
            "skos": skos_namespace,
            "xml": xml_namespace,
            "xsd": xsd_namespace,
            "doap": doap_namespace,
            "foaf": foaf_namespace,
            "rdfs": rdfs_namespace,
            "oboInOwl": oboInOwl_namespace,
            "oboLegacy": oboLegacy_namespace,            
        }
        graph = Graph()
        graph.parse(data=input_owl_string, format="xml", namespaces=namespaces)


        # Create a new graph for the result
        result_graph = Graph()
        # Replace namespace prefixes in the result graph to use the "canonical" ones
        for prefix, namespace in namespaces.items():
            result_graph.bind(prefix, namespace, replace=True)

        # Iterate through each class in the input graph
        for class_uri in graph.subjects(RDF.type, OWL.Class):
            class_element = URIRef(class_uri)

            # Add owl:Class triple
            result_graph.add((class_element, RDF.type, OWL.Class))

            # Add other properties to the owl:Class element
            for triple in graph.triples((class_uri, None, None)):
                result_graph.add((class_element, triple[1], triple[2]))

        # Create an XMLSerializer instance
        serializer = XMLSerializer(result_graph)

        # Use BytesIO to create an in-memory file-like object for bytes
        stream = BytesIO()

        # Serialize the modified graph to the in-memory stream
        serializer.serialize(stream)

        # Get the resulting string from the stream
        result_string = xml.dom.minidom.parseString(stream.getvalue()).toprettyxml()
        
        return result_string

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Perform reformatting and get the resulting OWL string
result_owl_string = create_owl_elements(input_owl_string)

if result_owl_string is not None:
    print(result_owl_string)
else:
    print("Error processing OWL data.")



<?xml version="1.0" ?>
<rdf:RDF xmlns="http://edamontology.org/" xmlns:oboInOwl="http://www.geneontology.org/formats/oboInOwl#" xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
	
  
	<rdf:Description rdf:about="http://edamontology.org/data_0581">
		
    
		<rdf:type rdf:resource="http://www.w3.org/2002/07/owl#Class"/>
		
    
		<rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#DeprecatedClass"/>
		
    
		<created_in>beta12orEarlier</created_in>
		
    
		<obsolete_since>beta12orEarlier</obsolete_since>
		
    
		<oldParent rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
		
    
		<oboInOwl:hasDefinition>A digital data archive typically based around a relational model but sometimes using an object-oriented, tree or graph-based model.</oboInOwl:hasDefinition>
		
    
		<oboInOwl:inSubset rdf:resource="http://edamontology.org/obsolete"/>
		
    
		<oboInOwl:replacedBy rdf